In [ ]:
!pip install transformers
!pip install datasets
!pip install nltk

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 43.7 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 31.0 MB/s 
     |████████████████████████████████| 596 kB 45.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 306 kB 5.3 MB/s 
     |████████████████████████████████| 133 kB 47.9 MB/s 
     |████████████████████████████████| 1.1 MB 48.9 MB/s 
     |████████████████████████████████| 243 kB 52.5 MB/s 
     |████████████████████████████████| 160 kB 50.1 MB/s 
     |████████████████████████████████| 192 kB 52.5 MB/s 
     |████████████████████████████████| 271 kB 35.8 MB/s 


In [ ]:
import random
from IPython.display import display, HTML
import numpy as np
import pandas as pd

from datasets import load_dataset, DatasetDict, Dataset, ClassLabel
from datasets import load_metric

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

import tensorflow as tf
from nltk.corpus import wordnet as wn

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
import nltk

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/2IMP40/github-comments.csv', names=['labels', 'text'], header=0)
# df['labels'] = df['labels'].replace(['neutral','negative','positive'],[0,1,2])
df.to_csv('/content/drive/MyDrive/2IMP40/github-comments.csv', index=None)
df

In [ ]:
dataset_infos = {'api': {'path': '/content/drive/MyDrive/2IMP40/api-review.xls', 'num_labels': 3, 'sentence_key': 'text', 'label_key': 'label'},
                 'app': {'path': '/content/drive/MyDrive/2IMP40/app-review.csv', 'num_labels': 3, 'sentence_key': 'text', 'label_key': 'label'},
                 'code-review': {'path': '/content/drive/MyDrive/2IMP40/code-review.xlsx', 'num_labels': 3, 'sentence_key': 'text', 'label_key': 'label'},
                 'github': {'path': '/content/drive/MyDrive/2IMP40/github-comments.csv', 'num_labels': 3, 'sentence_key': 'Text', 'label_key': 'Polarity'},
                 'jira': {'path': '/content/drive/MyDrive/2IMP40/jira-issues.csv', 'num_labels': 3, 'sentence_key': 'text', 'label_key': 'label'},
                 'stackoverflow': {'path': '/content/drive/MyDrive/2IMP40/stackoverflow.csv', 'num_labels': 3, 'sentence_key': 'text', 'label_key': 'label'}
                 }

cache_dir = 'drive/MyDrive/2IMP40/cache'

base_model_checkpoint = 'albert-base-v2'
model_name = base_model_checkpoint.split("/")[-1]
metric_name = 'accuracy'

dataset_name = 'github'
num_labels = dataset_infos[dataset_name]['num_labels']
sentence_key = dataset_infos[dataset_name]['sentence_key']
label_key = dataset_infos[dataset_name]['label_key']

batch_size = 8
num_train_epochs = 2
task = f'{dataset_name}_task'

model_output_dir = f'{cache_dir}/{model_name}-finetuned-{task}'
model_save_dir = f'drive/MyDrive/2IMP40/{task}'

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

dataset = load_dataset('csv', data_files=dataset_infos[dataset_name]['path'])
show_random_elements(dataset['train'], num_examples=10)

Using custom data configuration default-ba97840ce3496908


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ba97840ce3496908/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

,labels,text
0,2,"@rafaelfranca Spot on. Thanks for the tip!"""
1,2,"typo here :)"""
2,2,"so nevermind.. this commit looks right, it's the original code that looks totally fishy"""
3,0,"@Vasago: i tried to maintain (if i didn't made any mistake) the same restriction they had, with some changes. if a command .xxx has admin rights but .xxx yyy had player rights, now both are limited to admin. Before it was also useless to have player right"""
4,0,"Single-line if-statements should be wrapped in curly-braces. Spacing between if and opening parenthesis is missing"""
5,2,"Cute name :)"""
6,1,"Made make_resourceful break :( If I understand it correctly I'm supposed to write <tt>class.some_attr</tt> instead of <tt>class..read_inheritable_attribute(:some_attr)</tt>"""
7,1,"@chrisrhoden He told me so... @DouweM But he did not mean to harm anybody, such things can happen by accident..."""
8,2,"+1, thanks for catching this"""
9,1,"@evgeniy123 use paperclip 2.3.11 — no leaks for me @sikachu really, something wrong with 2.4.2 — eating memory as a monster :-( But only when cache_classes = false"""


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# 90% train, 10% validation
dataset = dataset['train'].train_test_split(test_size=0.1)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, cache_dir=cache_dir)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['test']

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
def compute_metrics(eval_pred):
    metric1 = load_metric("accuracy")
    metric2 = load_metric("recall")
    metric3 = load_metric("precision")
    metric4 = load_metric("f1")
    metric5 = load_metric("matthews_correlation")
    
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = metric1.compute(predictions=predictions, references=labels)['accuracy'] 
    recall = metric2.compute(predictions=predictions, references=labels, average='micro')['recall'] 
    precision = metric3.compute(predictions=predictions, references=labels, average='micro')['precision'] 
    f1 = metric4.compute(predictions=predictions, references=labels, average="micro")['f1'] 
    matthews_correlation = metric5.compute(predictions=predictions, references=labels)['matthews_correlation'] 
    return {'accuracy' : accuracy, 'recall': recall, 'precision': precision, 'f1': f1, 'matthews_correlation':  matthews_correlation}


model = AutoModelForSequenceClassification.from_pretrained(base_model_checkpoint, num_labels=num_labels, cache_dir=cache_dir)

training_args = TrainingArguments(
    output_dir=model_output_dir,
    evaluation_strategy="epoch",
    save_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model=metric_name,
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.bias', 'predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

In [ ]:
trainer.train()

In [ ]:
### EVALUATE AND SAVE THE BEST ONE
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 379
  Batch size = 8


{'eval_accuracy': 0.3641160949868074,
 'eval_f1': 0.3027162286021934,
 'eval_loss': 1.0927389860153198,
 'eval_matthews_correlation': 0.05776469458845873,
 'eval_precision': 0.320634575330884,
 'eval_recall': 0.3641160949868074,
 'eval_runtime': 32.519,
 'eval_samples_per_second': 11.655,
 'eval_steps_per_second': 1.476}

In [ ]:
from os import path, listdir, makedirs

### SAVE THE TRAINED MODEL
makedirs(model_save_dir)
model.save_pretrained(model_save_dir)

Configuration saved in drive/MyDrive/github_task/config.json
Model weights saved in drive/MyDrive/github_task/pytorch_model.bin


## LOAD THE FINETUNED MODEL

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/github_task", num_labels=num_labels, cache_dir=cache_dir)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

loading configuration file /content/drive/MyDrive/github_task/config.json
Model config AlbertConfig {
  "_name_or_path": "/content/drive/MyDrive/github_task",
  "architectures": [
    "AlbertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute"

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 713
  Batch size = 8


{'eval_accuracy': 0.9074333800841514,
 'eval_f1': 0.9074333800841514,
 'eval_loss': 0.4261937737464905,
 'eval_matthews_correlation': 0.8583487895962311,
 'eval_precision': 0.9074333800841514,
 'eval_recall': 0.9074333800841514,
 'eval_runtime': 52.4406,
 'eval_samples_per_second': 13.596,
 'eval_steps_per_second': 1.716}

In [ ]:
eval_predictions = trainer.predict(eval_dataset)

The following columns in the test set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 713
  Batch size = 8


In [ ]:
id2label = {0: "neutral", 1: "negative", 2: "positive"}

logits = tf.nn.softmax(eval_predictions.predictions)
label_ids = np.argmax(logits, 1)
labels = map(lambda x: id2label[x], label_ids)
labels = list(labels)

true_labels = pd.DataFrame(eval_dataset)['labels']
true_labels = list(map(lambda x: id2label[x], true_labels)) 
print(labels)
print(true_labels)

print(classification_report(true_labels, labels))

['positive', 'positive', 'negative', 'neutral', 'negative', 'negative', 'neutral', 'negative', 'positive', 'positive', 'negative', 'negative', 'neutral', 'neutral', 'negative', 'positive', 'negative', 'neutral', 'positive', 'neutral', 'positive', 'negative', 'negative', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'negative', 'positive', 'negative', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'negative', 'negative', 'negative', 'negative', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'negative', 'positive', 'neutral', 'positive', 'positive', 'positive', 'negative', 'neutral', 'positive', 'neutral', 'negative', 'positive', 'negative', 'negative', 'neutral', 'neutral', 'neutral', 'negative', 'positive', 'positive', 'neutral', 'neutral', 'positive', 'negative', 'positive', 'negative', 'negative', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'positive', 'neutral', 'positive', 'negativ

# PREDICTION STARTS HERE

In [ ]:
nltk.download('words')
nltk.download('punkt')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
dirty_path = '/content/drive/MyDrive/2IMP40/issues_data.csv'
clean_path = '/content/drive/MyDrive/2IMP40/issues_data_clean.csv'
issues_df_dirty = pd.read_csv(dirty_path)

In [ ]:
issues_df_working = issues_df_dirty.dropna()

lower = str.upper
isalpha = str.isalpha
word_tokenize = nltk.word_tokenize
join = " ".join

def is_english_word(word):
  if isalpha(word):
    return True
  
  if lower(word) in words:
    return True

  return False

def remove_non_english_from_sentence(str):
  words = word_tokenize(str)
  words = filter(is_english_word, words)
  return join(words)

titles = list(issues_df_working['title'])
descs = list(issues_df_working['desc'])

titles = list(map(remove_non_english_from_sentence, titles))
descs = list(map(remove_non_english_from_sentence, descs))

issues_df_working['title'] =  titles
issues_df_working['desc'] = descs

issues_df_working = issues_df_working.dropna()
issues_df_working.to_csv(clean_path, index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
issues_df = pd.read_csv(clean_path)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, cache_dir=cache_dir)

def tokenize_function_2_sentence(examples):
    return tokenizer(examples["title"], examples['desc'], padding="max_length", truncation=True)

dataset = Dataset.from_pandas(issues_df)
print(dataset)
tokenized_datasets = dataset.map(tokenize_function_2_sentence, batched=True)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/albert-base-v2/resolve/main/config.json from cache at drive/MyDrive/2IMP40/cache/e48be00f755a5f765e36a32885e8d6a573081df3321c9e19428d12abadf7dba2.b8f28145885741cf994c0e8a97b724f6c974460c297002145e48e511d2496e88
Model config AlbertConfig {
  "_name_or_path": "albert-base-v2",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers"

Dataset({
    features: ['_id', 'title', 'desc'],
    num_rows: 117812
})


  0%|          | 0/118 [00:00<?, ?ba/s]

In [ ]:
show_random_elements(dataset, num_examples=10)

,_id,title,desc
0,5d92f0c1afc58313293a8c42,Would it be possible to add rel tag dropdown with attributes to hyperlink window,When I try to add a hyperlink to an anchor word there is only option to the target attribute of the HTML element It would be good to be able to add rel attribute too tag nofollow bookmark etc I am struggling to add nofollows by hand Thank you Tamas
1,5b351dd36e373d64706f047f,Date Conversion doesnt use client Locale,Datetime values are not correctly converted when the client locale is different from server locale Example Client locale pt portuguese The date time format here is Server locale en english When a form is submitted with the value december according pt Locale struts returns a conversion error Invalid field value for field XXXXX When a form is submitted with the value February according pt Locale no input error but the value in the ValueStack will be May month and day INVERTED I tried to put the parameter with pt and and values but the same error
2,58c8d9b81cf3f06cf1590230,Incorrect directory path resolve when running from a different directory,Suppose you have two projects in two directories Each one of them have its own When you run Ant on from evrything is ok When you run from ant relative pathes are based on projB by Ivy whereas the correct way as Ant and Maven works is to use projA as the base directory assuming has basedir attribute In other words Ivy should take the base directory as specified in the file in project tag in basedir attribute and calculate it based on the location of the file not based on the current directory where Ant was invoced from
3,5f0d69665e36a95000e26980,Better Fault Tolerance,Other queue technologies provide a manner of fault tolerance missing from AMQ message semantics That is messages can be acknowledged at any time by a client Failing to do so within the messages TTL should result in the message on the queue so another client can it Reliable messaging with AMQ currently pertains to only message receipt but in practical systems distributing work via a queue this is unsufficient semantics to ensure tolerance of faults during work processing In that case clients will only acknowledge a message in the event of successful processing of that message left to the client to decide If the client were to suffer a fatality during processing the work associated with the message is left undone in the current AMQ because it can not be In these extreme but not uncommon fault conditions it is not possible for the client to the message Combining TTL behavior in the Broker and on the client of messages should achieve the desired increase in described here
4,5b76a003e1867c5bbd9a48a7,PATCH remove ignored memory allocation,code allocates an object and then allocates another object and stores it into the same reference thereby rendering the first allocation useless
5,5b62db81dddc135844b55605,Support index format creation,Archiva should provide an option to generate the indices for the nexus indexer which is used by
6,5b475e4dc298bd5a8f82479d,Ca run individual unit tests for plugins in nutch,I can run ant test in the directory but it runs all unit tests and I just want to test a single plugin Previous ways posted online for running a single unit test do work including ant test in the plugin directory I get BUILD FAILED Problem failed to create task or type antlib settings using with the junit argument junit In it says Class not found In src it says Error Could not find or load main class
7,5b3b31d5b79d66130de0dd17,Pig should not crash when it encounters an unknown function during interactive use,I get this The crash is unfortunate when all that happened is that I should have capitalized count grunt z foreach y generate group count y Function count not found at at at at at at at at at at at at at at at at at at at
8,5f0d67655e36a95000e24968,Creating temporary destinations omits security checkings,for example will successfully create a temporary topic even though it is not allowed by secu

In [ ]:
prediction_output = trainer.predict(tokenized_datasets)

The following columns in the test set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: desc, _id, title.
***** Running Prediction *****
  Num examples = 117812
  Batch size = 8


In [ ]:
id2label = {0: "neutral", 1: "negative", 2: "positive"}

logits = tf.nn.softmax(prediction_output.predictions)
label_ids = np.argmax(logits, 1)
labels = map(lambda x: id2label[x], label_ids)
labels = list(labels)
print(labels)
issues_df['sentiment'] = labels

labeled_path = '/content/drive/MyDrive/2IMP40/issues_data_labeled.csv'
issues_df.to_csv(labeled_path, index=False)

['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neu

In [ ]:
issues_df['sentiment'].value_counts()

neutral     112664
positive      3715
negative      1433
Name: sentiment, dtype: int64